# Face Verification Across Multiple Videos
This notebook extracts faces from multiple videos, generates face embeddings, and checks if the same face appears in all videos.

In [1]:
!pip install opencv-python deepface mtcnn numpy scipy

In [2]:
import cv2
import os
import numpy as np
from deepface import DeepFace
from mtcnn import MTCNN
from scipy.spatial.distance import cosine

detector = MTCNN()

## Step 1: Extract Frames from Videos

In [3]:
def extract_frames(video_path, output_folder, frame_interval=30):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    success, frame = cap.read()
    
    while success:
        if frame_count % frame_interval == 0:
            frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
        success, frame = cap.read()
        frame_count += 1
    cap.release()
    return output_folder

## Step 2: Detect Faces in Extracted Frames

In [4]:
def detect_faces(image_path):
    image = cv2.imread(image_path)
    faces = detector.detect_faces(image)
    
    if faces:
        x, y, width, height = faces[0]['box']
        face_img = image[y:y+height, x:x+width]
        return face_img
    return None

## Step 3: Extract Face Embeddings using DeepFace

In [5]:
def get_face_embedding(face_img):
    temp_path = "temp_face.jpg"
    cv2.imwrite(temp_path, face_img)
    try:
        embedding = DeepFace.represent(img_path=temp_path, model_name="ArcFace")[0]["embedding"]
        return np.array(embedding)
    except Exception as e:
        print(f"Embedding generation failed: {e}")
        return None

## Step 4: Compare Faces Across Videos

In [6]:
def is_same_person(embedding1, embedding2, threshold=0.5):
    if embedding1 is None or embedding2 is None:
        return False
    return cosine(embedding1, embedding2) < threshold

## Step 5: Full Pipeline for Multiple Videos

In [7]:
def process_videos(video_paths):
    reference_embedding = None
    results = {}
    
    for idx, video_path in enumerate(video_paths):
        print(f"Processing {video_path}...")
        
        output_folder = f"frames_video_{idx}"
        extract_frames(video_path, output_folder)
        
        frame_files = sorted(os.listdir(output_folder))
        detected_face = None
        
        for frame in frame_files:
            face_img = detect_faces(os.path.join(output_folder, frame))
            if face_img is not None:
                detected_face = face_img
                break
        
        if detected_face is None:
            results[video_path] = "No face detected"
            continue
        
        embedding = get_face_embedding(detected_face)
        
        if embedding is None:
            results[video_path] = "Face detected, but embedding not generated"
            continue
        
        if reference_embedding is None:
            reference_embedding = embedding
            results[video_path] = "Reference face set"
        else:
            if is_same_person(reference_embedding, embedding):
                results[video_path] = "Same face detected"
            else:
                results[video_path] = "Different face detected"
    
    return results

## Run the Model

In [9]:
video_list = ["Face Recognition/Test1.mov", "Face Recognition/Test2.mov", "Face Recognition/Test3.mov","Face Recognition/Test4.mov"]

verification_results = process_videos(video_list)

for video, result in verification_results.items():
    print(f"{video}: {result}")

Processing Face Recognition/Test1.mov...
Processing Face Recognition/Test2.mov...
Processing Face Recognition/Test3.mov...
Processing Face Recognition/Test4.mov...
Face Recognition/Test1.mov: Reference face set
Face Recognition/Test2.mov: Different face detected
Face Recognition/Test3.mov: Same face detected
Face Recognition/Test4.mov: Same face detected
